In [ ]:
import pickle

with open("tmp/id2topics_240205.pkl", "rb") as f:
    id2topics = pickle.load(f)

In [ ]:
# selected = {k: v for k, v in id2topics.items() if "geoarchive" in v}
selected = {k: v for k, v in id2topics.items() if "criticalmaas" in v}
len(cm)

In [ ]:
with open("tmp/id2topics.pkl", "wb") as f:
    pickle.dump(selected, f)

In [ ]:
from askem.elastic import get_text

In [ ]:
with open("tmp/ingested.pkl", "rb") as f:
    ingested = pickle.load(f)

In [ ]:
ingesting = {k: v for k, v in cm.items() if k not in ingested}

In [ ]:
ingesting

In [ ]:
ingesting = {k: get_text(k) for k in cm.keys() if k not in ingested}

In [ ]:
y = get_text("620f2eb6ad0e9c819b108ea9")
y

Compare ingested status

In [ ]:
from askem.utils import get_id_topics_from_weaviate
import weaviate
from dotenv import load_dotenv
import os
import logging

load_dotenv()


def get_client(url: str = None, apikey: str = None) -> weaviate.Client:
    """Get a weaviate client."""

    if url is None:
        url = os.getenv("WEAVIATE_URL")

    if apikey is None:
        apikey = os.getenv("WEAVIATE_APIKEY")

    logging.info(f"Connecting to Weaviate at {url}")
    return weaviate.Client(url, weaviate.auth.AuthApiKey(apikey))


client = get_client()
in_weaviate = get_id_topics_from_weaviate(client)

In [ ]:
import pickle

with open("tmp/id2topics_240205.pkl", "rb") as f:
    in_elastic = pickle.load(f)

with open("tmp/id2topics_weaviate.pkl", "rb") as f:
    in_weaviate = pickle.load(f)

In [ ]:
# Venn sets
all_ids = set(in_elastic.keys()) | set(in_weaviate.keys())

not_in_weaviate = all_ids - set(in_weaviate.keys())
intersection_set = set(in_elastic.keys()) & set(in_weaviate.keys())
not_in_elastic = all_ids - set(in_elastic.keys())

print(
    f"intersection: {len(intersection_set)}, not_in_weaviate: {len(not_in_weaviate)}, not_in_elastic: {len(not_in_elastic)}"
)

Check if all topics align

In [ ]:
from random import choice
from askem.elastic import get_text

In [ ]:
for x in intersection_set:
    assert in_elastic[x] == in_weaviate[x]

In [ ]:
x = choice(list(not_in_weaviate))
get_text(x)

Parse error.log to get empty ids

In [ ]:
import pickle
from askem.ingest_v2 import parse_error_log

# Load empty ids from file
with open("tmp/empty_ids.pkl", "rb") as f:
    empty_ids = pickle.load(f)

print(len(empty_ids))

In [ ]:
# Add new empty ids from error log
parsed = parse_error_log()

for k, v in parsed.items():
    print(f"{k}: {len(v)}")

empty_ids.extend(parsed["empty"])
empty_ids = list(set(empty_ids))
print(len(empty_ids))

In [ ]:
# Write to file
with open("tmp/empty_ids.pkl", "wb") as f:
    pickle.dump(empty_ids, f)